In [ ]:
messages = [
    {"role": "user", "content": "Hi there!"},
    {"role": "assistant", "content": "Nice to meet you!"},
    {"role": "user", "content": "Can I ask a question?"}
]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

name = 

tokenizer = AutoTokenizer.from_pretrained(name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
model = AutoModelForCausalLM.from_pretrained(name, device_map='cuda')

In [ ]:
instructions = [
    ({'role': 'system', 'content': ''},
    {'role': 'user', 'content': 'I hate cats!!!!!'},
    {'role': 'assistant', 'content': 'You are so stupid!!!! Cats are so cute!!!!!!'}) for i in range(len(examples[query]))
]

texts = tokenizer.apply_chat_template(
    instructions,
    tokenize=False
)